# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            print(f)
            all_files.append(os.path.abspath(f))    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')
#single_file=next((join(filepath, f) for f in os.listdir(filepath) if isfile(join(filepath, f))), 
#                  "default value here")
print(song_files)

data/song_data/A/A/C/TRAACER128F4290F96.json
data/song_data/A/A/C/TRAACPE128F421C1B9.json
data/song_data/A/A/C/TRAACHN128F1489601.json
data/song_data/A/A/C/TRAACOW128F933E35F.json
data/song_data/A/A/C/TRAACCG128F92E8A55.json
data/song_data/A/A/C/TRAACTB12903CAAF15.json
data/song_data/A/A/C/TRAACSL128F93462F4.json
data/song_data/A/A/C/TRAACFV128F935E50B.json
data/song_data/A/A/C/TRAACNS128F14A2DF5.json
data/song_data/A/A/C/TRAACVS128E078BE39.json
data/song_data/A/A/C/TRAACIW12903CC0F6D.json
data/song_data/A/A/C/TRAACLV128F427E123.json
data/song_data/A/A/C/TRAACQT128F9331780.json
data/song_data/A/A/C/TRAACZK128F4243829.json
data/song_data/A/A/C/.ipynb_checkpoints/TRAACFV128F935E50B-checkpoint.json
data/song_data/A/A/C/.ipynb_checkpoints/TRAACZK128F4243829-checkpoint.json
data/song_data/A/A/C/.ipynb_checkpoints/TRAACCG128F92E8A55-checkpoint.json
data/song_data/A/A/A/TRAAAVG12903CFA543.json
data/song_data/A/A/A/TRAAAFD128F92F423A.json
data/song_data/A/A/A/TRAAAEF128F4273421.json
data/song_

In [5]:
filepath=song_files[0]
print(filepath)

/home/workspace/data/song_data/A/A/C/TRAACER128F4290F96.json


In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
print(df)
df=df.iloc[0]
song_df=df[['song_id', 'title', 'artist_id', 'year', 'duration']]

unfmt_song_data=song_df.values.tolist()
song_data=[]
for item in unfmt_song_data:
    if isinstance(item,float):
        song_data.append(round(item,4))
    else:
        song_data.append(item)
print(song_data)


            artist_id  artist_latitude    artist_location  artist_longitude  \
0  ARMAC4T1187FB3FA4C         40.82624  Morris Plains, NJ         -74.47995   

                 artist_name   duration  num_songs             song_id  \
0  The Dillinger Escape Plan  207.77751          1  SOBBUGU12A8C13E95D   

                             title  year  
0  Setting Fire to Sleeping Giants  2004  
['SOBBUGU12A8C13E95D', 'Setting Fire to Sleeping Giants', 'ARMAC4T1187FB3FA4C', 2004, 207.7775]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
print(song_table_insert)
print(song_data)
print(cur.execute(song_table_insert, song_data))
conn.commit()

INSERT INTO songs (song_id,title,artist_id ,year,duration) VALUES (%s,%s,%s,%s,%s) ON CONFLICT DO NOTHING 
['SOBBUGU12A8C13E95D', 'Setting Fire to Sleeping Giants', 'ARMAC4T1187FB3FA4C', 2004, 207.7775]
None


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_df =df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
print(artist_df)
artist_data=artist_df.values.tolist()
print(artist_data)

artist_id                  ARMAC4T1187FB3FA4C
artist_name         The Dillinger Escape Plan
artist_location             Morris Plains, NJ
artist_latitude                       40.8262
artist_longitude                       -74.48
Name: 0, dtype: object
['ARMAC4T1187FB3FA4C', 'The Dillinger Escape Plan', 'Morris Plains, NJ', 40.826239999999999, -74.479950000000002]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')

data/log_data/2018/11/2018-11-12-events.json
data/log_data/2018/11/2018-11-18-events.json
data/log_data/2018/11/2018-11-04-events.json
data/log_data/2018/11/2018-11-11-events.json
data/log_data/2018/11/2018-11-27-events.json
data/log_data/2018/11/2018-11-29-events.json
data/log_data/2018/11/2018-11-05-events.json
data/log_data/2018/11/2018-11-15-events.json
data/log_data/2018/11/2018-11-23-events.json
data/log_data/2018/11/2018-11-14-events.json
data/log_data/2018/11/2018-11-13-events.json
data/log_data/2018/11/2018-11-06-events.json
data/log_data/2018/11/2018-11-08-events.json
data/log_data/2018/11/2018-11-20-events.json
data/log_data/2018/11/2018-11-02-events.json
data/log_data/2018/11/2018-11-07-events.json
data/log_data/2018/11/2018-11-17-events.json
data/log_data/2018/11/2018-11-28-events.json
data/log_data/2018/11/2018-11-30-events.json
data/log_data/2018/11/2018-11-24-events.json
data/log_data/2018/11/2018-11-26-events.json
data/log_data/2018/11/2018-11-10-events.json
data/log_d

In [12]:
filepath =log_files[15] 

In [13]:
df =pd.read_json(filepath, lines=True)
print(df)

                                    artist        auth firstName gender  \
0                             Miami Horror   Logged In      Kate      F   
1                        The White Stripes   Logged In      Kate      F   
2                             Juan Carmona   Logged In      Kate      F   
3            Alison Krauss / Union Station   Logged In      Kate      F   
4                  Bullet For My Valentine   Logged In      Kate      F   
5                                 BjÃÂ¶rk   Logged In      Kate      F   
6                               Kanye West   Logged In      Kate      F   
7                            Amy Winehouse   Logged In      Ayla      F   
8                                   Saosin   Logged In      Kate      F   
9                                 Gorillaz   Logged In      Kate      F   
10   World Inferno/Friendship Society_ The   Logged In      Kate      F   
11                            Fall Out Boy   Logged In      Kate      F   
12                       

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df =pd.read_json(filepath, lines=True)
df=df.loc[df['page'] == 'NextSong']
user_df=pd.DataFrame(df)

df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df.head()
t=df.copy()

In [15]:
t =df.copy()

In [22]:
#time_data=(t.ts,t.ts.dt.hour,t.ts.dt.day,t.ts.dt.dayofweek,t.ts.dt.month,t.ts.dt.year,t.ts.dt.weekday)
    #Create the dictionary to map field names and  time data.
time_data=[t.ts,t.ts.dt.hour,t.ts.dt.day,t.ts.dt.dayofweek,t.ts.dt.month,t.ts.dt.year,t.ts.dt.weekday]
time_labels=["start_time","hour","day","week","month","year","weekday"]
time_dict=dict(zip(time_labels, time_data))
print(time_dict)
   
""""time_dict={"start_time":t.ts,
               "hour":t.ts.dt.hour,
               "day":t.ts.dt.day, 
               "week":t.ts.dt.dayofweek, 
               "month":t.ts.dt.month,
               "year":t.ts.dt.year,
               "weekday":t.ts.dt.weekday}"""


{'start_time': 0     2018-11-07 00:01:16.796
1     2018-11-07 00:05:26.796
2     2018-11-07 00:09:27.796
3     2018-11-07 00:14:58.796
4     2018-11-07 00:17:49.796
5     2018-11-07 00:21:44.796
6     2018-11-07 00:27:32.796
7     2018-11-07 00:28:00.796
8     2018-11-07 00:30:50.796
9     2018-11-07 00:34:48.796
10    2018-11-07 00:38:03.796
11    2018-11-07 00:40:08.796
12    2018-11-07 00:43:00.796
13    2018-11-07 00:46:01.796
15    2018-11-07 00:49:34.796
16    2018-11-07 00:52:29.796
20    2018-11-07 01:31:24.796
21    2018-11-07 01:35:00.796
22    2018-11-07 01:39:24.796
23    2018-11-07 01:42:43.796
24    2018-11-07 01:46:23.796
25    2018-11-07 02:53:30.796
26    2018-11-07 02:56:26.796
27    2018-11-07 03:00:28.796
28    2018-11-07 03:01:00.796
29    2018-11-07 03:03:28.796
30    2018-11-07 03:09:03.796
31    2018-11-07 03:12:44.796
33    2018-11-07 05:27:54.796
34    2018-11-07 05:30:16.796
                ...          
169   2018-11-07 19:25:25.796
170   2018-11-07 19:28:42

'"time_dict={"start_time":t.ts,\n               "hour":t.ts.dt.hour,\n               "day":t.ts.dt.day, \n               "week":t.ts.dt.dayofweek, \n               "month":t.ts.dt.month,\n               "year":t.ts.dt.year,\n               "weekday":t.ts.dt.weekday}'

In [23]:
time_df= pd.DataFrame.from_dict(time_dict)
time_df.head()

,start_time,hour,day,week,month,yearweekday
0,2018-11-07 00:01:16.796,0,7,2,11,2018
1,2018-11-07 00:05:26.796,0,7,2,11,2018
2,2018-11-07 00:09:27.796,0,7,2,11,2018
3,2018-11-07 00:14:58.796,0,7,2,11,2018
4,2018-11-07 00:17:49.796,0,7,2,11,2018


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
    print(list(row))
    cur.execute(time_table_insert, list(row))
    conn.commit()

IndexError: list index out of range

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [33]:
user_df = user_df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df=user_df.drop_duplicates(keep='first').reset_index(drop=True)
print(user_df)

   userId   firstName  lastName gender level
0      97        Kate   Harrell      F  paid
1      63        Ayla   Johnson      F  free
2       8      Kaylee   Summers      F  free
3       9       Wyatt     Scott      M  free
4      89     Kynnedi   Sanchez      F  free
5      50         Ava  Robinson      F  free
6     101      Jayden       Fox      M  free
7      14    Theodore    Harris      M  free
8      10      Sylvie      Cruz      F  free
9     100       Adler   Barrera      M  free
10     80       Tegan    Levine      F  free
11     86       Aiden      Hess      M  free
12     66       Kevin  Arellano      M  free
13     44      Aleena     Kirby      F  paid
14     15        Lily      Koch      F  paid
15     23      Morris   Gilmore      M  free
16     29  Jacqueline     Lynch      F  free
17     84     Shakira      Hunt      F  free
18      2     Jizelle  Benjamin      F  free
19     26        Ryan     Smith      M  free
20     33     Bronson    Harris      M  free
21     52 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [34]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [35]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    #print(song_select)
    #print(row.song,row.length)
    #len=round(row.length,5)
    print(len)
    print(cur.execute(song_select, (str(row.song),str(row.artist),float(row.length))))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    
    print(results)
    # insert songplay record
    songplay_data = [str(row.ts),str(row.userId),str(row.level),songid,artistid,str(row.sessionId),str(row.location),str(row.userAgent)]
    print(songplay_data)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

<built-in function len>
None
None
['2018-11-07 00:01:16.796000', '97', 'paid', None, None, '293', 'Lansing-East Lansing, MI']
<built-in function len>
None
None
['2018-11-07 00:05:26.796000', '97', 'paid', None, None, '293', 'Lansing-East Lansing, MI']
<built-in function len>
None
None
['2018-11-07 00:09:27.796000', '97', 'paid', None, None, '293', 'Lansing-East Lansing, MI']
<built-in function len>
None
None
['2018-11-07 00:14:58.796000', '97', 'paid', None, None, '293', 'Lansing-East Lansing, MI']
<built-in function len>
None
None
['2018-11-07 00:17:49.796000', '97', 'paid', None, None, '293', 'Lansing-East Lansing, MI']
<built-in function len>
None
None
['2018-11-07 00:21:44.796000', '97', 'paid', None, None, '293', 'Lansing-East Lansing, MI']
<built-in function len>
None
None
['2018-11-07 00:27:32.796000', '97', 'paid', None, None, '293', 'Lansing-East Lansing, MI']
<built-in function len>
None
None
['2018-11-07 00:28:00.796000', '63', 'free', None, None, '223', 'Santa Rosa, CA']
<b

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [36]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.